# Load needed modules

In [1]:
# modules:
import pandas as pd
import os
from openai import OpenAI
import numpy as np

import re

import math

from langchain_openai import ChatOpenAI

# to read pdf files
from langchain_community.document_loaders import PyPDFLoader

# to read text of pdf files
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# get external variables or functions:
import src.API_key as key

# Load data to feed ChatGPT

In [2]:
## set working environment
#> Get the current working directory
print(os.getcwd())
directory = os.getcwd()

# List files in the current working directory
files = os.listdir('.')
# Display the list of files
print(files)

c:\DATEN\PHD\Article_SoftRobotIntervention\Analyses\main study - ChatGPT
['.venv', 'data', 'output', 'runChatGPT.ipynb', 'src', 'v01', 'v02']


## Load Scenario Texts
This should be the final scenario texts in English of the two robots:

* rescue robot
* socially assistive robot


source: https://python.langchain.com/v0.2/docs/tutorials/pdf_qa/

In [3]:
# Load the scenario texts of the rescue robot
file_path = directory + "/data/scenario texts/" + "rescue robot" + ".pdf"
loader = PyPDFLoader(file_path)
doc_RR = loader.load()
print(len(doc_RR))

# Load the scenario texts of the socially assistive robot
file_path = directory + "/data/scenario texts/" + "socially assistive robot" + ".pdf"
loader = PyPDFLoader(file_path)
doc_SAR = loader.load()
print(len(doc_SAR))

6
6


 Using a text splitter, the loaded documents will be split into smaller documents that can more easily fit into an LLM's context window, then load them into a vector store. Then a retriever from the vector store is created for use in our RAG chain.

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits_RR = text_splitter.split_documents(doc_RR)
vectorstore_RR = Chroma.from_documents(documents=splits_RR, embedding=OpenAIEmbeddings(openai_api_key=key.openai_api_key))

retriever_RR = vectorstore_RR.as_retriever()

In [5]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=key.openai_api_key)

In [20]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever_RR, question_answer_chain)

results = rag_chain.invoke({"input": "What are the advantages of rescue robots?"})

results

{'input': 'What are the advantages of rescue robots?',
 'context': [Document(metadata={'page': 0, 'source': 'c:\\DATEN\\PHD\\Article_SoftRobotIntervention\\Analyses\\main study - ChatGPT/data/scenario texts/rescue robot.pdf'}, page_content='essential supplies, and autonomously assisting in the rescue of victims, these robots \ncan enhance the efficiency and effectiveness of rescue operations.  \nWhile robots f or search and rescue are still in the development phase, it is important \nto consider the ethical aspects (risks and benefits) of these technologies.'),
  Document(metadata={'page': 0, 'source': 'c:\\DATEN\\PHD\\Article_SoftRobotIntervention\\Analyses\\main study - ChatGPT/data/scenario texts/rescue robot.pdf'}, page_content='––––––––––––––––––  First Page in Experiment ––––––––––––––––––   \nPlease read the following text carefully.  \nAfterwards we will ask you to draw a CAM around the predefined \nknotpoint:  \n"Which risks and benefits come to your mind when considering the 

In [22]:
print(results["context"][0].page_content)
print(results["context"][0].metadata)

essential supplies, and autonomously assisting in the rescue of victims, these robots 
can enhance the efficiency and effectiveness of rescue operations.  
While robots f or search and rescue are still in the development phase, it is important 
to consider the ethical aspects (risks and benefits) of these technologies.
{'page': 0, 'source': 'c:\\DATEN\\PHD\\Article_SoftRobotIntervention\\Analyses\\main study - ChatGPT/data/scenario texts/rescue robot.pdf'}


## Load .xlsx files (lists of words)
This should be the final scenario texts in English of the two robots combined and seperated:

* rescue robot_multipleSheets
* socially assistive robot_multipleSheets
* rescue robot_socially assistive robot_multipleSheets

sources:
* https://python.langchain.com/v0.2/docs/tutorials/llm_chain/ (Build a Simple LLM Application with LCEL)
* https://python.langchain.com/v0.2/docs/how_to/structured_output/ (How to return structured data from a model)
* https://python.langchain.com/v0.2/docs/how_to/llm_token_usage_tracking/ (How to track token usage for LLMs)


In [5]:
## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "rescue robot_socially assistive robot_multipleSheets" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names combined:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_Combined = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}


## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "rescue robot_multipleSheets" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names RR:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_RR = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}



## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "socially assistive robot_multipleSheets" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names SAR:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_SAR = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}


Sheet names combined: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']
Sheet names RR: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']
Sheet names SAR: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']


In [6]:
# Example: Access data from a specific sheet
sheet_name = 'T'
print(all_sheets_Combined[sheet_name].shape)
print(all_sheets_RR[sheet_name].shape)
print(all_sheets_SAR[sheet_name].shape)

(88, 6)
(41, 6)
(47, 6)


Use dictionaries to map each word to its comment.

In [7]:
def create_multivalue_dict(df, key_col, value_col):
    """
    Create a dictionary from a DataFrame where each key maps to a list of values.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    key_col (str): The column name to be used as keys.
    value_col (str): The column name to be used as values.
    
    Returns:
    dict: A dictionary where each key maps to a list of values.
    """
    # Remove rows with NaN in the key columns
    df = df.dropna(subset=[key_col])

    # Create a dictionary to map items to their comments, allowing for multiple comments per key
    multivalue_dict = {}
    for key, value in zip(df[key_col], df[value_col]):
        if key in multivalue_dict:
            multivalue_dict[key].append(value)
        else:
            multivalue_dict[key] = [value]

    return multivalue_dict

# Example usage
data = {
    'constant': ['a', 'b', 'a', 'c', 'b', np.nan],
    'constant_comments': ['comment1', 'comment2', 'comment3', 'comment4', 'comment5', 'comment6']
}
df = pd.DataFrame(data)

df_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
print(df_mapping)

{'a': ['comment1', 'comment3'], 'b': ['comment2', 'comment5'], 'c': ['comment4']}


In [9]:
# Remove rows with NaN in the key columns as they cannot be used as dictionary keys
#> not sensitive to multiple identical keys: dict(zip(df['constant'], df['constant_comments']))
df = all_sheets_Combined[sheet_name]

constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
print("mapping constant x comments:", constant_comments_mapping)
print(len(constant_comments_mapping))

new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
print("mapping new x comments:", new_comments_mapping)
print(len(new_comments_mapping))

mapping constant x comments {'constant availability (timewise)': [nan], 'unconditional emotional accompaniment': [nan], 'discreet': ['Stop complaining'], 'Less abuse weak': ['There is unspeakable abuse of vulnerable individuals by nursing staff. I read a newspaper article where a person had installed a hidden camera in the room of their elderly mother (to find out who was stealing), and it turned out there was severe, severe abuse by nursing staff.'], 'People think': [nan], 'exact task repetition': [nan], 'robot not sick': [nan], 'neutral': ['Social assistance robots are neutral and do not swing from emotional state to positive or negative'], 'objective': ["No condemnation or falsification of feelings. Soberly considers the facts. // The robot doesn't care who you are."], 'fact-based': ['Assistance robots proceed schematically according to facts and can refer to scientific data free from emotions. // Scientific elaboration and programming of behaviors can achieve a desired reaction. Sc

# Set up ChatGPT

In [24]:
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)


result = prompt_template.invoke({"language": "italian", "text": "hi"})

print("result:", result)
print("result.to_messages():", result.to_messages())

model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=key.openai_api_key)

chain = prompt_template | model



with get_openai_callback() as cb:
    result = chain.invoke({"language": "german", "text": "How much is the fish?"})
    print(result)
    print("---")
print()


print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

result: messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')]
result.to_messages(): [SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')]
content='Wie viel kostet der Fisch?' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 23, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-e0ca7b80-b0e7-4e79-b88e-3b00e7f31150-0' usage_metadata={'input_tokens': 23, 'output_tokens': 8, 'total_tokens': 31}
---

Total Tokens: 31
Prompt Tokens: 23
Completion Tokens: 8
Total Cost (USD): $5.0499999999999994e-05


In [25]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'Wie viel kostet der Fisch?'

In [31]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key=key.openai_api_key)


from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats and dogs in the space")

Joke(setup='Why did the cat and dog go to space?', punchline='To find the purr-fect planet!', rating=8)

In [32]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'To keep an eye on the mouse!',
 'rating': 8}

In [34]:
system_template = "Translate the following into the following two languages: {language1}, {language2}:"
user_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)


result = prompt_template.invoke({"language1": "italian", "text": "hi"})

print("result:", result)
print("result.to_messages():", result.to_messages())

model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=key.openai_api_key)

chain = prompt_template | model



with get_openai_callback() as cb:
    result = chain.invoke({"language1": "german", "text": "How much is the fish?"})
    print(result)
    print("---")
print()


print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

KeyError: "Input to ChatPromptTemplate is missing variables {'language2', 'language1'}.  Expected: ['language1', 'language2', 'text'] Received: ['language', 'text']"

## 1. create prompt text

In [7]:
# Function to create the dynamic text
def create_prompt_learning_report(data):
    reports = []
    shown_names = set()
    for index, row in data.iterrows():
        name = row['Name']
        if name not in shown_names:
            shown_names.add(name)
            oral_mark_wording_inv = row['OralMark_wording_inv']
            oral_mark_wording_int = row['OralMark_wording_int']
            
            if pd.isna(oral_mark_wording_inv):
                reports.append(None)
            else:                
                condition = data['Name'] == name
                data_checks = data[condition]

                # Construct the text
                report = f"""Ich möchte einen Lernentwicklungsbericht für einen Schüler namens {name} schreiben. Verwende hierbei die dritte Person, um die Lernentwicklung des Schülers sachlich zu beschreiben. Achte hierbei auf das Geschlecht des Namens. Verwende nur den Vornamen des Schülers. Dieser Lernentwicklungsbericht sollte nur aus einem einzigen Paragraphen bestehen und folgend aufgebaut sein:"""
                report += f""" 1. Schreibe ein Satz zur mündlichen Note im Unterricht, ohne das Wort "mündlich" zu verwenden. 
                2. Schreibe zu jeder Kontrolle zwei bis drei Sätze, wobei das Niveau des Schülers am Anfang genannt wird. Beachte hierbei zu jedem Textblock der jeweiligen Kontrolle etwas zu schreiben, wobei "UND" für verschiedene Fähigkeiten innerhalb einer erworbenen Kompetenz steht. Schreibe pro Kontrolle / Check jedoch nicht mehr als 40 Worte. Verwende nicht das Wort "Kontrolle" oder "Check". 
                3. Am Schluss des Paragraphen soll "(Fe)" stehen."""
                report += f"""
                Hierbei übergebe ich dir im Folgenden, die dafür relevanten Informationen:     
                """
                report += f"""zu 1. mündliche Note:  
                gezeigtes Engagement im Unterricht: {oral_mark_wording_inv}, sowie gezeigtes Interesse im Unterricht: {oral_mark_wording_int}.
                """
                report += f"""zu 2. Kontrollen:"""
                
                for index, row in data_checks.iterrows():
                    check = row['Check']
                    competences = re.sub('\xad', '', row['competences'])
                    wording = row['wording']
                    report += f"""
                    {check}:
                    Niveau der  erworbene Kompetenzen: {wording}

                    Kompetenzen:
                    {competences}
                    
                    """
                    
                reports.append(report)
    return reports

promptText = create_prompt_learning_report(df)
print(promptText)

[None, None, 'Ich möchte einen Lernentwicklungsbericht für einen Schüler namens Carbone, Valerio schreiben. Verwende hierbei die dritte Person, um die Lernentwicklung des Schülers sachlich zu beschreiben. Achte hierbei auf das Geschlecht des Namens. Verwende nur den Vornamen des Schülers. Dieser Lernentwicklungsbericht sollte nur aus einem einzigen Paragraphen bestehen und folgend aufgebaut sein: 1. Schreibe ein Satz zur mündlichen Note im Unterricht, ohne das Wort "mündlich" zu verwenden. \n                2. Schreibe zu jeder Kontrolle zwei bis drei Sätze, wobei das Niveau des Schülers am Anfang genannt wird. Beachte hierbei zu jedem Textblock der jeweiligen Kontrolle etwas zu schreiben, wobei "UND" für verschiedene Fähigkeiten innerhalb einer erworbenen Kompetenz steht. Schreibe pro Kontrolle / Check jedoch nicht mehr als 40 Worte. Verwende nicht das Wort "Kontrolle" oder "Check". \n                3. Am Schluss des Paragraphen soll "(Fe)" stehen.\n                Hierbei übergebe i

## 2. apply ChatGPT

### overall

Function for API call

In [8]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback
def basic_API_call(
    openai_api_key,
    template,
    model_name="gpt-4",
    max_tokens=150,
):

    prompt = PromptTemplate(template=template)
    # top_p_value = float("0")
    seed = 123
    llm_chain = LLMChain(
        prompt=prompt,
        llm=ChatOpenAI(
            # llm=OpenAI(
            temperature=0.0,
            openai_api_key=openai_api_key,
            model_name=model_name,
            max_tokens=max_tokens,
            # model_kwargs={"top_p": top_p_value, "seed": seed},
            model_kwargs={"seed": seed},
        ),
        return_final_only=False,
    )
    
    with get_openai_callback() as cb:
        response = llm_chain.invoke(
            {}
        )
        print(cb)
        
    return response

In [9]:
# response = basic_API_call(key.openai_api_key, promptText[16], model_name="gpt-4", max_tokens=1000)
# response['text']
# response

add LEBs to existing pupil data.frame

In [10]:
# load data set of pupils
df_wide = pd.read_excel(directory + "/" + nameData_wide + ".xlsx")
# Add an empty column "LEB_neu" to the DataFrame
df_wide['LEB_neu'] = None

c:\Users\fenn\AppData\Local\R-MINI~1\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
response_full = []

for i in range(0, len(promptText)):
    if promptText[i] is not None:
        # print(i)
        # print(promptText[i])
        response = basic_API_call(key.openai_api_key, promptText[i], model_name="gpt-4", max_tokens=1000)
            
        df_wide['LEB_neu'][i] = response['text']
        response_full.append(response)

c:\Users\fenn\AppData\Local\R-MINI~1\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Tokens Used: 965
	Prompt Tokens: 674
	Completion Tokens: 291
Successful Requests: 1
Total Cost (USD): $0.037680000000000005


C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

Tokens Used: 1242
	Prompt Tokens: 880
	Completion Tokens: 362
Successful Requests: 1
Total Cost (USD): $0.048119999999999996


C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

...

C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

Tokens Used: 1212
	Prompt Tokens: 877
	Completion Tokens: 335
Successful Requests: 1
Total Cost (USD): $0.04641


C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

Tokens Used: 1058
	Prompt Tokens: 672
	Completion Tokens: 386
Successful Requests: 1
Total Cost (USD): $0.04332


C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

In [12]:
print(os.getcwd())
os.chdir("../output") 
print(os.getcwd())
# set the current working directory
directory = os.getcwd()
# Save the DataFrame to an Excel file
df_wide.to_excel(directory + "/" + nameData_wide + "_LEB" + ".xlsx", index=False)


# Save the DataFrame to an Excel file
df_json = pd.DataFrame(response_full)
df_json.to_excel(directory + "/" + nameData_wide + "_fullResponse" + ".xlsx", index=False)

c:\DATEN\PHD\Steffen_LEB\LEB generieren\data
c:\DATEN\PHD\Steffen_LEB\LEB generieren\output
